# Catan Board Visualization

Visualization of the Catan board with NetworkX graph structure.

## Setup

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import RegularPolygon, Circle
import numpy as np
import catan_env

%matplotlib inline

## Visualization Functions

Works with graph-based BoardState using NetworkX.

In [ ]:
# Resource colors
RESOURCE_COLORS = {
    'wood': '#228B22',
    'brick': '#8B4513',
    'wheat': '#FFD700',
    'sheep': '#90EE90',
    'ore': '#708090',
    'desert': '#F4A460',
}

# Player colors
PLAYER_COLORS = ['#FF4444', '#4444FF', '#FFAA00', '#44FF44']

# Hex geometry
HEX_RADIUS = 1.0
HEX_WIDTH = np.sqrt(3) * HEX_RADIUS
HEX_HEIGHT = 2 * HEX_RADIUS


def get_tile_center(tile) -> tuple[float, float]:
    """Convert tile to 2D coordinates."""
    row, col = tile.row, tile.col
    y = -row * (HEX_HEIGHT * 0.75)
    row_width = catan_env.ROW_SIZES[row]
    row_offset = -(row_width - 1) * HEX_WIDTH / 2
    x = row_offset + col * HEX_WIDTH
    return x, y


def get_vertex_position(board, vertex_id: int) -> tuple[float, float]:
    """Get vertex position using proper hex geometry (at corners)."""
    if vertex_id not in board.graph:
        return (0, 0)
    
    adjacent_tiles = board.graph.nodes[vertex_id]['adjacent_tiles']
    if not adjacent_tiles:
        return (0, 0)
    
    tile_id = adjacent_tiles[0]
    tile = board.tiles[tile_id]
    tile_center_x, tile_center_y = get_tile_center(tile)
    
    tile_vertices = board.get_vertices_for_tile(tile_id)
    try:
        vertex_index = tile_vertices.index(vertex_id)
    except ValueError:
        tile_centers = [get_tile_center(board.tiles[tid]) for tid in adjacent_tiles]
        return (sum(x for x, y in tile_centers) / len(tile_centers),
                sum(y for x, y in tile_centers) / len(tile_centers))
    
    angles = [np.pi/2, np.pi/6, -np.pi/6, -np.pi/2, -5*np.pi/6, 5*np.pi/6]
    angle = angles[vertex_index]
    return (tile_center_x + HEX_RADIUS * np.cos(angle),
            tile_center_y + HEX_RADIUS * np.sin(angle))


def plot_board(board, figsize=(14, 12)) -> None:
    """Plot Catan board with hexes, roads, settlements, and cities."""
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_aspect('equal')
    
    # Draw tiles
    for tile in board.tiles:
        center_x, center_y = get_tile_center(tile)
        color = RESOURCE_COLORS.get(tile.resource, '#CCCCCC')
        
        hexagon = RegularPolygon(
            (center_x, center_y), numVertices=6, radius=HEX_RADIUS,
            orientation=np.pi/6, facecolor=color, edgecolor='black',
            linewidth=2, alpha=0.7
        )
        ax.add_patch(hexagon)
        
        if tile.token is not None:
            circle = Circle((center_x, center_y), radius=0.35,
                          facecolor='white', edgecolor='black',
                          linewidth=1.5, zorder=3)
            ax.add_patch(circle)
            
            number_color = 'red' if tile.token in [6, 8] else 'black'
            ax.text(center_x, center_y, str(tile.token),
                   ha='center', va='center', fontsize=16,
                   fontweight='bold', color=number_color, zorder=4)
    
    # Draw roads (from graph edges)
    for v_a, v_b, data in board.graph.edges(data=True):
        if data['owner'] is not None:
            x1, y1 = get_vertex_position(board, v_a)
            x2, y2 = get_vertex_position(board, v_b)
            color = PLAYER_COLORS[data['owner'] % len(PLAYER_COLORS)]
            ax.plot([x1, x2], [y1, y2], color=color, linewidth=5,
                   solid_capstyle='round', zorder=5)
    
    # Draw settlements and cities (from graph nodes)
    for vertex_id, data in board.graph.nodes(data=True):
        if data['owner'] is not None:
            x, y = get_vertex_position(board, vertex_id)
            color = PLAYER_COLORS[data['owner'] % len(PLAYER_COLORS)]
            
            if data['is_city']:
                ax.scatter(x, y, s=500, marker='s', color=color,
                          edgecolors='black', linewidths=2.5, zorder=6)
            else:
                ax.scatter(x, y, s=250, marker='o', color=color,
                          edgecolors='black', linewidths=2.5, zorder=6)
    
    # Styling
    ax.set_xlim(-6, 6)
    ax.set_ylim(-8, 2.5)
    ax.axis('off')
    ax.set_title('Catan Board', fontsize=20, fontweight='bold', pad=20)
    
    # Legend
    resource_patches = [
        mpatches.Patch(color=color, label=resource.capitalize())
        for resource, color in RESOURCE_COLORS.items()
    ]
    
    active_players = set()
    for _, data in board.graph.nodes(data=True):
        if data['owner'] is not None:
            active_players.add(data['owner'])
    for _, _, data in board.graph.edges(data=True):
        if data['owner'] is not None:
            active_players.add(data['owner'])
    
    player_patches = [
        mpatches.Patch(color=PLAYER_COLORS[p % len(PLAYER_COLORS)], label=f'Player {p}')
        for p in sorted(active_players)
    ]
    
    all_patches = resource_patches
    if player_patches:
        all_patches.append(mpatches.Patch(color='white', label=''))
        all_patches.extend(player_patches)
    
    ax.legend(handles=all_patches, loc='upper left',
             bbox_to_anchor=(1.02, 1), fontsize=10)
    
    plt.tight_layout()
    plt.show()

## Example 1: Empty Board

In [ ]:
# Create a random board
board = catan_env.generate_random_board()
plot_board(board)

## Example 2: Board with Game Pieces

Using the graph API to place pieces.

In [ ]:
# Create board
board = catan_env.generate_random_board()

# Player 0: settlements and roads
board.place_settlement(0, 10)
board.place_settlement(0, 30)

# Get edges using graph API
edges_v10 = list(board.graph.edges(10))
if edges_v10:
    board.place_road(0, edge=edges_v10[0])
    if len(edges_v10) > 1:
        board.place_road(0, edge=edges_v10[1])

# Upgrade to city
board.upgrade_to_city(0, 10)

# Player 1
board.place_settlement(1, 20)
board.place_settlement(1, 40)
edges_v20 = list(board.graph.edges(20))
if edges_v20:
    board.place_road(1, edge=edges_v20[0])

# Player 2
board.place_settlement(2, 5)

plot_board(board)

## Example 3: Graph Operations

Demonstrating NetworkX graph features.

In [ ]:
board = catan_env.generate_random_board()

# Place settlement
vertex_id = 10
board.place_settlement(0, vertex_id)

# Get neighbors using graph
neighbors = list(board.graph.neighbors(vertex_id))
print(f"Vertex {vertex_id} has {len(neighbors)} neighbors: {neighbors}")

# Try to place at adjacent vertex (should fail - distance rule)
if neighbors:
    result = board.place_settlement(1, neighbors[0])
    print(f"Placing at adjacent vertex {neighbors[0]}: {result} (should be False)")

# Place road
edges = list(board.graph.edges(vertex_id))
if edges:
    print(f"\nEdges at vertex {vertex_id}: {edges}")
    board.place_road(0, edge=edges[0])
    print(f"Placed road on edge {edges[0]}")

# Show vertex data
print(f"\nVertex {vertex_id} data: {board.graph.nodes[vertex_id]}")

plot_board(board)

## Graph Structure Benefits

The NetworkX graph enables:
- Clean placement logic with `graph.neighbors()`
- Access to 100+ graph algorithms
- Easy feature extraction for RL
- Strategic analysis (connectivity, paths, etc.)

See `graph_demo.py` for more examples!